In [1]:
%%bash
pip install numpy-quaternion

In [2]:
import numpy as np
import quaternion

import cv2

import time

from copy import deepcopy

from IPython.display import clear_output

/usr/local/lib/python2.7/dist-packages/quaternion/numba_wrapper.py:29: UserWarning: 

!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
Could not import from either numbapro or numba.
This means that the code will run MUCH more slowly.
You probably REALLY want to install numba / numbapro.
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!

  warnings.warn(warning_text)


In [12]:
from fast_image import Image
from color import Color
from feat_2d import Circle2D
from feat_3d import Sphere3D
from searchers import do_linear_search, do_binary_search
from optimizers import findPose
from ros import RosHandler

In [13]:
%load_ext line_profiler

# Configure Jupyter to auto-reload constituent libraries when reasonable
%load_ext autoreload
%autoreload 2
%aimport -rospy

The line_profiler extension is already loaded. To reload it, use:
  %reload_ext line_profiler
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Useful resources

[OpenCV tutorial](http://docs.opencv.org/3.0-beta/doc/py_tutorials/py_gui/py_video_display/py_video_display.html#playing-video-from-file)

![](http://wiki.ros.org/image_pipeline/CameraInfo?action=AttachFile&do=get&target=CameraCoords.png)

# Define hardcoded map for testing tracking

In [14]:
# Define colors
green = Color(minimum=(30, 50, 100), maximum=(60, 255, 255))
orange = Color(minimum=(5, 120, 130), maximum=(30, 255, 255))
blue = Color(minimum=(90, 60, 90), maximum=(150, 255, 255))
black = Color(minimum=(0, 0, 0), maximum=(255, 210, 150))

# Define 3D features
greenSphere = Sphere3D((.55/2,.45/2,0), .106, green)
orangeSphere = Sphere3D((-.55/2,.45/2,0), .106, orange)
blueSphere = Sphere3D((.55/2,-.45/2,0), .106, blue)
blackSphere = Sphere3D((-.55/2,-.45/2,0), .106, black)

# Define initial condition
startPos = np.array([0, 0, -1.5])
startQuat = np.quaternion(1, 0, 0 ,0)

# ROS functionality

In [15]:
ros = RosHandler()

# Capture and handle video

In [16]:
FILE = '../test_data/unstabilized.mp4'
cv2.namedWindow('raw_frame')

In [17]:
cap = cv2.VideoCapture(FILE)

def handleFrame(frame, features, verbose=False):
    image = Image(frame)

    for f in features:
        f.refine(image, verbose=verbose)
    
    return image


def main():
    pos = startPos
    quat = startQuat
#     pos = np.array([-1, 0, -1])
#     quat = np.quaternion(1,0,.5,0).normalized()
    spheres = deepcopy([greenSphere, orangeSphere, blueSphere, blackSphere])
    i=0
    while cap.isOpened():
        verbose = i%50 == 0
        if verbose:
            clear_output()
        i += 1
        
        startTime = time.time()
        ret, frame = cap.read()
        readTime = time.time()

        if frame is None or len(frame) <= 0:
            print "End of video file reached"
            break

        # Do work here
        features = [sphere.project(pos, quaternion.as_rotation_matrix(quat))
                   for sphere in spheres]
        startFeats = deepcopy(features)
        
        image = handleFrame(frame, features, verbose=verbose)
        
        pos, quat = findPose(spheres, features, pos, quat, verbose=verbose)
#         quat = np.quaternion(1,0,0,0)
            
        finishTime = time.time()
        
        ros.publish_pose(pos, quat)

        for f in features:
            f.draw(frame)
            
        image.draw(frame)
        scaled = cv2.resize(frame, dsize=None, fx=0.5, fy=0.5)
        cv2.imshow('raw_frame', scaled)
        
        drawnTime = time.time()

        cv2.waitKey(1)
        
        if verbose:
            processingTime = (finishTime-readTime)
            print "{0:.1f}ms reading and drawing image, {1:.1f}ms ({3}fps) processing, {2} pixels accessed\n".format(
                (readTime-startTime + drawnTime-finishTime)*1000, 
                processingTime*1000, 
                len(image.pixelsAccessed),
                int(1/processingTime))
        
main()

17 evaluations needed, final error [  1.45482061e+00   1.86975397e-01  -5.96127875e-01   1.11654353e-01
  -1.61669463e+00   6.02760075e-01   8.01122917e-01  -9.99450086e-01
  -2.54850687e-05   0.00000000e+00   0.00000000e+00   0.00000000e+00
   0.00000000e+00   0.00000000e+00   0.00000000e+00]
18.1ms reading and drawing image, 20.2ms (49fps) processing, 148 pixels accessed

End of video file reached


# Profile things

In [9]:
cap = cv2.VideoCapture(FILE)

greenSmall = greenSphere.project(
    pos=startPos, 
    orientation=quaternion.as_rotation_matrix(startQuat))
import scipy.optimize

def bench():
    features = deepcopy([greenSmall])
    for _ in range(60):
        ret, frame = cap.read()
        handleFrame(frame, features)
    
%lprun -r -f do_binary_search bench()

NameError: name 'greenSphere' is not defined